# Avoided crossing with asymmetric quantum well

[Go back to index](./index.ipynb)

[https://en.wikipedia.org/wiki/Avoided_crossing](https://en.wikipedia.org/wiki/Avoided_crossing)

In quantum physics and quantum chemistry, an avoided crossing is the phenomenon where two eigenvalues of an Hermitian matrix representing a quantum observable and depending on N continuous real parameters cannot become equal in value ("cross") except on a manifold of N-2 dimensions.

In this widget, we showed a asymmetric quantum well potential:

$x^4 - 0.6x^2 + \mu x$

$\mu$ is the potential parameter. By changing the $\mu$ value, we can see how the tow lowest eigenvalues change, which demonstrates the avoided crossing phenomenon.

In [36]:
%matplotlib widget
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros, concatenate
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt
from math import pi
from numpy import *

colors = ['b', 'r', 'g', 'c', 'm', 'y'] 

def potential(x, mu):
    """Potential function for double trough, mu: Parameter"""
    return x**4 - 0.6*x**2 + mu*x # asymmetrical double trough

def diagonalisierung(hquer, L, N, pot=potential, mu = 0.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         mu: potential function parameter 
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (arry of length N)
         dx: grid space
         V:  Potential at positions x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # grid points 
    dx = x[1] - x[0]                              # grid spacing
    V = pot(x, mu)
    z = hquer**2 /2.0/dx**2                       
    h = (diag(V+2.0*z) + diag(-z*ones(N-1), -1)   
                      + diag(-z*ones(N-1), 1) )   #  Hamilton-Matrix

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                  # real part of the eigenvalues
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sort the ew by ind
    ef = ef[:, ind]                               # Sort the columns
                                                
    ef = ef/sqrt(dx)                              # Correct standardization
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.2, fak= 5.0):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    fak = fak/100.0; 
    
    ax[1].plot(x, V, c='k', linewidth=1.3)  
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.01, Emax])
    
    #ax[1].yaxis.set_label_position('right')
    #ax[1].yaxis.tick_right()
    ax[1].set_xticks([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax[1].set_xticklabels([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
      
    indmax = sum(ew<=Emax)                       
    
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[1].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])


def plot_zeitentwicklung(ax, ew, ef, x, V, coeff, zeiten, fak): 
    """ Calculate the time evolution of a wave packet.
    """
    
    E0_qm = dot(abs(coeff)**2, ew)                       # qm. Energy expectation
    fak = fak/100.0                                      # Plot scaling factor
    phi_t0 = dot(ef, coeff*exp(-1j*ew*zeiten[0]/hquer))  # Time evolution
                                                         # Plot with line width
                                                         
    phi = dot(ef, coeff*exp(-1j*ew*zeiten[-1]/hquer))
    ax[1].plot(x, E0_qm+fak*abs(phi)**2, 'r--', linewidth=2.0)


In [38]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

mu = 0.06                                            # Potentialparameter
L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.


smu = FloatSlider(value = 0.06, min = -0.1, max = 0.1, step = 0.01, description = r'$\mu$: ')
                                                         
fig, ax = plt.subplots(2, 1, figsize=(8, 8))
fig.canvas.header_visible = False

fig.suptitle('Numerial Solution of One Dimension Schroedinger Equation', fontsize = 10)

mu1 = []
ew1 = []
ew2 = []


for i in concatenate((linspace(-0.1, -0.01, 10), linspace(-0.01, 0.01, 20), linspace(0.01, 0.1, 10))):
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = i)
    mu1.append(i)
    ew1.append(ew[0])
    ew2.append(ew[1])

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = mu)

ax[0].plot(mu1, ew1, c='b')
ax[0].plot(mu1, ew2, c='r')

s2 = ax[0].plot(mu, ew[1], 'ro', label = str(format(ew[1], '.3f')))
s1 = ax[0].plot(mu, ew[0], 'bo', label = str(format(ew[0], '.3f')))

ax[0].legend()

ax[0].set_xlim([min(mu1), max(mu1)])
ax[0].set_ylim([min(ew1), max(ew2)+0.005])

ax[0].set_xlabel(r'Potential parameter $\mu$ value', fontsize = 10)
ax[0].set_ylabel(r'The values of the two lowest eigenvalues', fontsize = 10)
ax[0].set_xticks([-0.1, -0.05, 0.0, 0.05, 0.1])
ax[0].set_xticklabels([-0.1, -0.05, 0.0, 0.05, 0.1])

plot_eigenfunktionen(ax, ew, ef, x, V)

sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')

def on_mu_change(change):
    global ew, ef, x, dx, V
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = smu.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)
    
    ax[0].lines.pop(-1)
    ax[0].lines.pop(-1)
    ax[0].plot(smu.value, ew[1], 'ro', label = str(format(ew[1], '.3f')))
    ax[0].plot(smu.value, ew[0], 'bo', label = str(format(ew[0], '.3f')))
    ax[0].legend()

smu.observe(on_mu_change, names = 'value')

def on_press(event):
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)

    plot_eigenfunktionen(ax, ew, ef, x, V)
    phi0 = (2.0*pi*sigma_x**2)**(-0.25)*exp(-(x-event.xdata)**2.0/(4.0*sigma_x**2))
    coeff = dot(conjugate(transpose(ef)), phi0)*dx
    plot_zeitentwicklung(ax, ew, ef, x, V, coeff, zeiten, fak = sfak.value)
    
def on_xfak_change(change):
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)


cid = fig.canvas.mpl_connect('button_press_event', on_press)

sfak.observe(on_xfak_change, names = 'value')

display(HBox([smu, sfak]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>